In [1]:
from my_parser import *
import my_parser
my_parser.tail_error

[]

In [64]:
foo = lambda x:x
foo.__name__

'<lambda>'

## calc recursive

In [3]:
calc_recursive = Parser()
calc_recursive.add_bnf('num','?=[0-9]+/*s:int(s)*/')
calc_recursive.add_bnf('rterm',"$(num)|'('$(sum)')'")
calc_recursive.add_bnf('term',
'''$(rterm) 
|?= ${t:rterm} '*' ${n:term} /* t,n:t*n */ 
|?= ${t:rterm} '/' ${n:term} /* t,n:t/n */ 
''')
calc_recursive.add_bnf('sum',
'''$(term)
|?= ${t:term} '+' ${n:sum} /* t,n:t+n */ 
|?= ${t:term} '-' ${n:sum} /* t,n:t-n */ 
''')
calc_recursive.main = 'sum'

In [4]:
calc_recursive.parse('2+2*2')

6

In [5]:
calc_recursive.debug = False
calc_recursive.parse('(2+2)*2')

8

In [6]:
calc_recursive.debug = True
calc_recursive.parse('2+2+2+3+5+2+2+2+3+5')

28

## calc cycle

In [7]:
calc_cyclic = Parser()
calc_cyclic = Parser()
calc_cyclic.add_bnf('num','?=[0-9]+/*s:int(s)*/')
calc_cyclic.add_bnf('rterm',"$(num)|'('$(sum)')'")

In [8]:
@calc_cyclic.add_bnf_seqfun('term','?= ${head:rterm} {?tail= * }((?s=["*/"]) ${x:rterm})')
def seqsum(head,tail):
    for o in tail:
        #print(o)
        if o['s']=='*': head*=o['x']
        if o['s']=='/': head/=o['x']
    return head
@calc_cyclic.add_bnf_seqfun('sum','?= ${head:term} {?tail= * }((?s=["+-"]) ${x:term})')
def seqsum(head,tail):
    for o in tail:
        #print(o)
        if o['s']=='+': head+=o['x']
        if o['s']=='-': head-=o['x']
    return head
calc_cyclic.main = 'sum'

In [9]:
calc_cyclic.parse('2')

2

In [10]:
calc_cyclic.parse('2+2')

4

In [11]:
calc_cyclic.parse('2+2*2')

6

In [12]:
calc_cyclic.parse('(2+2)*2')

8

In [13]:
calc_cyclic.debug = True
calc_cyclic.parse('2+2+2+3+5+2+2+2+3+5')

28

## JSON

In [14]:
#test = Parser()
#test.add_bnf('character',r'''?= [^"\x0-\x19\"\\"]  
#| "\\" (?=escape)''')



In [15]:
my_parser.debug = False
json_p = Parser(main = 'json')
json_p.add_bnf('json','$(element)')
json_p.add_bnf('value','''$(object)|$(array)|$(string)|$(number)
               |?="true"/*x:True*/|?="false"/*x:False*/|?="null"/*x:None*/''')
json_p.add_bnf('object',"?= '{' ws '}'/*x:{}*/|'{' $(members) '}'")
json_p.add_bnf('members',"""?= $(member)/*mms:{mms["k"]:mms["v"]}*/
               |?= ${m=member} ',' ${mms=members}/*m,mms:{m["k"]:m["v"],**mms}*/""")
json_p.add_bnf('member',"ws ${k=string} ws ':' ${v=element}")

json_p.add_bnf('array',"?= '[' ws ']'/*x:[]*/|'[' $(elements) ']'")
json_p.add_bnf('elements',"?=$(element)/*x:[x]*/|?= ${e=element} ',' ${els=elements} /*e,els:[e,*els]*/")
json_p.add_bnf('element',"ws $(value) ws")

json_p.add_bnf('string',"'\"' (?=characters) '\"'")
json_p.add_bnf('characters','""|character characters')

json_p.add_bnf('character',r'''?= [^\x00-\x19\"\\]  
| "\\" (?=escape)'''
              )

my_parser.debug = False
json_p.add_bnf('escape',r'''?=[\"\\\/]
|?="b"/*x:'\b'*/
|?="f"/*x:'\f'*/
|?="n"/*x:'\n'*/
|?="r"/*x:'\r'*/
|?="t"/*x:'\t'*/
|'u' (?=hex hex hex hex/*x:chr(int(x,16))*/) '''
              )
json_p.add_bnf('hex',"digit|[a-fA-F]")

json_p.add_bnf('number','''?= ${i=integer} (?f=fraction exponent)
/*i,f: float(i+f) if f!="" else int(i)*/''')
json_p.add_bnf('integer','digit|onenine digits|"-" digit|"-" onenine digits')
json_p.add_bnf('digits','digit|digit digits')
json_p.add_bnf('digit','[0-9]')
json_p.add_bnf('onenine','[1-9]')
json_p.add_bnf('fraction','""|"." digits')
json_p.add_bnf('exponent','""|"E" sign digits|"e" sign digits')
json_p.add_bnf('sign','["+-"]?')

json_p.add_bnf('ws','""|"\x20" ws|"\x0A" ws|"\x0D" ws|"\x09" ws')
               

In [16]:
ascii(33)

'33'

In [17]:
p_bnf_quoted_sequence('"\x00-\x19\"\\"',0)

(5, '\x00-\x19')

In [18]:
json_p.trace_log = False
json_p.parse('{}')

{}

In [19]:
json_p.parse('"qwer"')

'qwer'

In [20]:
json_p.trace_log = False
json_p.parse(r'"qw\u0011er"')

'qw\x11er'

In [21]:
json_p.trace_log = False
json_p.parse('[2]')

[2]

In [22]:
json_p.parse('[2,3]')

[2, 3]

In [23]:
json_p.trace_log = False
json_p.parse('{"a":2}')

{'a': 2}

In [24]:
json_p.trace_log = False
json_p.parse('{"a":2,"b":3}')

{'a': 2, 'b': 3}

In [25]:
json_p.parse(r'{"a":2,"b":[7,7.4,"\u0000"]}')

{'a': 2, 'b': [7, 7.4, '\x00']}

In [26]:
json_p.parse(r'{"a":2,"b":{"c":"d"}}')

{'a': 2, 'b': {'c': 'd'}}

## JSON cyclic

In [2]:
my_parser.debug = False
json_cp = Parser(main = 'json')
json_cp.add_bnf('json','$(element)')
json_cp.add_bnf('value','''$(object)|$(array)|$(string)|$(number)
               |?="true"/*x:True*/|?="false"/*x:False*/|?="null"/*x:None*/''')

@json_cp.add_bnf_seqfun('object',"?='{' {?a= *}($(member) ',') {?l= ?}$(member) '}'")
def make_obj(a,l):
    o = {}
    for lo in a:
        o[lo['k']]=lo['v']
    for lo in l:
        o[lo['k']]=lo['v']
    return o
json_cp.add_bnf('member',"ws ${k=string} ws ':' ${v=element}")

json_cp.add_bnf('array',"?= '[' {?a= *}($(element) ',') {?l= ?}$(element) ']' /*a,l:a+l*/")
json_cp.add_bnf('element',"ws $(value) ws")

json_cp.add_bnf('string',"'\"' (?=character*) '\"'")
json_cp.add_bnf('character',r'''?= [^\x00-\x19\"\\]  
| "\\" (?=escape)'''
              )
json_cp.add_bnf('escape',r'''?=[\"\\\/]
|?="b"/*x:'\b'*/
|?="f"/*x:'\f'*/
|?="n"/*x:'\n'*/
|?="r"/*x:'\r'*/
|?="t"/*x:'\t'*/
|'u' (?=hex hex hex hex/*x:chr(int(x,16))*/) '''
              )
json_cp.add_bnf('hex',"[0-9a-fA-F]")

json_cp.add_bnf('number','''?= ${i=integer} (?f=fraction exponent)
/*i,f: float(i+f) if f!="" else int(i)*/''')
json_cp.add_bnf('integer','"-"? [0-9] | "-"? [1-9] [0-9]+')
json_cp.add_bnf('fraction','""|"." [0-9]+')
json_cp.add_bnf('exponent','""|"E" ["+-"]? [0-9]+|"e" ["+-"]? [0-9]+')

json_cp.add_bnf('ws',r'[\x20\x0A\x0D\x09]*')
               

In [3]:
json_cp.trace_log = False
json_cp.parse('{}')

{}

In [4]:
json_cp.parse('"qwer"')

'qwer'

In [5]:
json_cp.trace_log = False
json_cp.parse(r'"qw\u0011er"')

'qw\x11er'

In [6]:
json_cp.trace_log = False
json_cp.parse('[2,]')

[2]

In [7]:
json_cp.parse('[2,3]')

[2, 3]

In [8]:
json_cp.trace_log = False
json_cp.parse('{"a":2}')

{'a': 2}

In [9]:
json_cp.trace_log = False
json_cp.parse('{"a":2,"b":3}')

{'a': 2, 'b': 3}

In [10]:
json_cp.parse(r'{"a":2,"b":[7,7.4,"\u0000"]}')

{'a': 2, 'b': [7, 7.4, '\x00']}

In [11]:
json_cp.parse(r'{"a":2,"b":{"c":"d"}}')

{'a': 2, 'b': {'c': 'd'}}

In [12]:
import codecs
import json
hash_path = r'D:\.files\patch 2020-01-02 02-42-28+0300 to 2020-01-05 02-40-35+0300.json'
with codecs.open(hash_path,'r', encoding='utf-8') as file:
    file_text = file.read()
    

In [13]:
#json_cp.trace_log = True
test_obj = json_cp.parse(file_text)

In [14]:
#json_cp.trace_log = True
real_obj = json.loads(file_text)

In [15]:
test_obj == real_obj

True